In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
import os
import spacy
from tqdm import tqdm
import xml.etree.ElementTree as et

In [3]:
import copy

def calc_precision(tp, fp):
    return tp/(tp + fp)

def calc_recall(tp, fn):
    return tp/(tp + fn)

def calc_fscore(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

def evaluate(gold_truth_labels, predictions):
    # Counts of true positives, false positives & false negatives
    tp, fp, fn = 0, 0, 0
    
    # List with false positives and false negatives
    fps, fns = [], []
    
    for gold, pred in zip(gold_truth_labels, predictions):
        
        tp_tmp, fp_tmp, fn_tmp, fns_temp, fps_temp  = evaluate_one_article(gold, pred)
        
        tp += tp_tmp
        fp += fp_tmp
        fn += fn_tmp
        
        fns.extend(fns_temp)
        fps.extend(fps_temp) 
        
    precision = calc_precision(tp, fp)
    recall = calc_recall(tp, fn)
    f_score = calc_fscore(precision, recall)    
    
    print(f'fp: {fp} | tp: {tp} | fn: {fn}')
    print(f'precision: {precision:.3f} | recall: {recall:.3f} | f-score: {f_score:.3f}')
    
    return fps, fns  
    

def evaluate_one_article(gold_truth, prediction):
    
    gold = gold_truth['entities'].copy()
    pred = prediction['entities'].copy()
    
    # Counts of true positives, false positives & false negatives
    tp, fp, fn = 0, 0, 0
    
    # List with false positives and false negatives
    fps, fns = [], []
    
    
    i = 0
    
    while len(gold) > 0 and len(pred) > 0:
        i += 1

        # Check if the first two elements are the same
        if gold[0] == pred[0]:
            tp += 1
            gold.pop(0)
            pred.pop(0)
        
        else:
            # Grab the first appearing element
            element, source = (gold[0], 'gold') if gold[0]['start_pos'] < pred[0]['start_pos'] else (pred[0], 'pred')
            
            # Remove the element first appearing element
            if source == 'gold':
                fn += 1
                fns.append(element['text'])
                gold.remove(element)
            elif source == 'pred':
                fp += 1
                fps.append(element['text'])
                pred.remove(element)
    
    if len(gold) > 0:
        fn += 1
    elif len(pred) > 0:
        fp += 1
        
    return tp, fp, fn, fns, fps       

def run_flair(text):

    # make a sentence
    sentence = Sentence(text)

    # run NER over sentence
    tagger.predict(sentence)
    
    for entity in sentence.to_dict(tag_type='ner')['entities']:
        print(entity)

In [4]:
def load_file(file_path):
    """
    Loads file and returns all the articles
    """
    # Load the data
    tree = et.parse(file_path)
    root = tree.getroot()

    return root

def process_article(article, filtered, file_path):
    """
    Takes article and process into desired structure
    """
    if 'GeoWebNews' in file_path:
        if filtered:
            return {'text': article.find('text').text, 
                    'entities': sorted([{'text': top.find('extractedName').text, 
                                                      'start_pos': int(top.find('start').text), 
                                                      'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym') 
                                                     if top.find('latitude') != None and top.find('longitude') != None], key=lambda k: k['start_pos'])}
        
        else:
            return {'text': article.find('text').text, 
                    'entities': sorted([{'text': top.find('extractedName').text, 
                                                      'start_pos': int(top.find('start').text), 
                                                      'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')], key=lambda k: k['start_pos'])}
    
    
    elif not filtered:
        return {'text': article.find('text').text,
                'entities': sorted([{'text': top.find('phrase').text,
                            'start_pos': int(top.find('start').text),
                            'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
                                 ], key=lambda k: k['start_pos'])}
        
    else:
        return {'text': article.find('text').text,
                'entities': sorted([{'text': top.find('phrase').text,
                            'start_pos': int(top.find('start').text),
                            'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
                             if top.find('gaztag/lat') != None and top.find('gaztag/lon') != None
                                 ], key=lambda k: k['start_pos'])}

def process_articles(root, filtered, file_path):
    """
    Takes articles and processes them into desired structure
    """
    data = []
    
    for article in root:
        
        data.append(process_article(article, filtered, file_path))
    
    return data

def prepare_data(file_path, filtered):
    
    root = load_file(file_path)
    
    data = process_articles(root, filtered, file_path)
    
    return data

In [5]:
def make_predictions_spacy(data):
    
    predictions = []
    
    texts = [article['text'] for article in data]
    
    for doc in tqdm(nlp.pipe(texts)):
        # Do something with the doc here
        
        pred = {'entities': [{'text': ent.text, 
                              'start_pos': len(doc[0:ent.end].text) - len(doc[ent.start]), 
                              'end_pos': len(doc[0:ent.end].text)} for ent in doc.ents if 
                                                             ent.label_ == 'LOC' or ent.label_ == 'GPE']}
        
        if pred:
            pred['text'] = doc
            
            predictions.append(pred)
        
    return predictions


### Load the default transfomer spacy model

In [6]:
# Only enable the ner tagger
nlp = spacy.load("en_core_web_lg", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

## TR-News

In [7]:
# Get file path LGL dataset
file_path = '../../data/TR-News/TR-News.xml'

In [8]:
data_all_toponyms = prepare_data(file_path, filtered=False)
data_filtered_toponyms = prepare_data(file_path, filtered=True)

#### Predictions for TR-News

In [9]:
predictions_all_toponyms = make_predictions_spacy(data_all_toponyms)
predictions_filtered_toponyms = make_predictions_spacy(data_filtered_toponyms)

118it [00:01, 64.98it/s]
118it [00:01, 70.72it/s]


#### Results TR-News & Comparison

In [10]:
# only toponyms w/ lat/long
fps, fns = evaluate(data_filtered_toponyms, predictions_filtered_toponyms)

fp: 256 | tp: 734 | fn: 515
precision: 0.741 | recall: 0.588 | f-score: 0.656


In [11]:
fps

['North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'Interstate 64',
 'the South Regional Jail',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'New York',
 'New York',
 'the United States',
 'the United States',
 'the United States',
 'the White House',
 'the Rose Garden',
 'Atlantic',
 'West Chester',
 'the United States',
 'Southern District',
 'New York',
 'South Carolina',
 'South Carolina',
 'New Milford',
 'Eastside',
 'London City',
 'London City',
 "San Francisco's",
 'Central\n            Anatolia',
 'the Golden Globe-',
 'the South Pacific',
 'the Venice International Film Festival',
 'Air Berlin',
 'the Central Coast',
 'West Coast',
 'Pearson International Airport',
 'Montreal',
 'Etobicoke',
 'Winnipeg',
 'Manitoba',
 'US',
 'Palma',
 'Little Portugal',
 'Dufferin Street',
 'Phoenix',
 'Phoenix',
 'Gatineau',
 'Ottawa',
 'Karak',
 'Karak Castle',
 'Town Hall',
 'La Liga',
 'the United States',
 'the White House',
 'New York

In [12]:
fns

['Turkish',
 'Aleppo',
 'Turkish',
 'Syrian',
 'Syrian',
 'U.S.',
 'Turkish',
 'Turkish',
 'Russian',
 'Syrian',
 'Russian',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Russian',
 'Turkish',
 'Russian',
 'Russian',
 'Russian',
 'Syrian',
 'North Carolina',
 'West Virginia',
 'Granville County',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'Granville County',
 'New York',
 'New York',
 'New York City',
 'United States',
 'United States',
 'United States',
 'White House',
 'Texas',
 'College Station',
 'Texas',
 'Texas',
 'U.S.',
 'West Chester',
 'Xavier University',
 'United States',
 'British',
 'European',
 'U.S.',
 'New York',
 'DETROIT',
 'U.S.',
 'U.S.',
 'United States',
 'New York',
 'Michigan',
 'Wisconsin',
 'South Carolina',
 'South Carolina',
 'Cuban',
 'BANTAM',
 'Danbury',
 'Bantam',
 'New Milford',
 'Danbury',
 'New Milford',
 'Calgary',

In [11]:
# all toponyms
fps, fns = evaluate(data_all_toponyms, predictions_all_toponyms)

fp: 256 | tp: 734 | fn: 555
precision: 0.741 | recall: 0.569 | f-score: 0.644


In [14]:
fps

['North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'Interstate 64',
 'the South Regional Jail',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'New York',
 'New York',
 'the United States',
 'the United States',
 'the United States',
 'the White House',
 'the Rose Garden',
 'Atlantic',
 'West Chester',
 'the United States',
 'Southern District',
 'New York',
 'South Carolina',
 'South Carolina',
 'New Milford',
 'Eastside',
 'London City',
 'London City',
 "San Francisco's",
 'Central\n            Anatolia',
 'the Golden Globe-',
 'the South Pacific',
 'the Venice International Film Festival',
 'Air Berlin',
 'the Central Coast',
 'West Coast',
 'Pearson International Airport',
 'Montreal',
 'Etobicoke',
 'Winnipeg',
 'Manitoba',
 'US',
 'Palma',
 'Little Portugal',
 'Dufferin Street',
 'Phoenix',
 'Phoenix',
 'Gatineau',
 'Ottawa',
 'Karak',
 'Karak Castle',
 'Town Hall',
 'La Liga',
 'the Stade de France',
 'the United States',
 'the 

In [15]:
fns

['Turkish',
 'Aleppo',
 'Turkish',
 'Syrian',
 'Syrian',
 'U.S.',
 'Turkish',
 'Kurdish',
 'Turkish',
 'Russian',
 'Syrian',
 'Russian',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Russian',
 'Turkish',
 'Russian',
 'Russian',
 'Russian',
 'Syrian',
 'North Carolina',
 'West Virginia',
 'Granville County',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'Granville County',
 'New York',
 'New York',
 'New York City',
 'United States',
 'United States',
 'United States',
 'White House',
 'Texas',
 'College Station',
 'Texas',
 'Texas',
 'U.S.',
 'West Chester',
 'Xavier University',
 'United States',
 'British',
 'European',
 'U.S.',
 'New York',
 'DETROIT',
 'U.S.',
 'U.S.',
 'United States',
 'New York',
 'Michigan',
 'Wisconsin',
 'South Carolina',
 'South Carolina',
 'Cuban',
 'BANTAM',
 'Danbury',
 'Bantam',
 'New Milford',
 'Danbury',
 'New Milford',

## LGL

In [12]:
# Get file path LGL dataset
file_path = '../../data/LGL/LGL.xml'

In [13]:
data_all_toponyms = prepare_data(file_path, filtered=False)
data_filtered_toponyms = prepare_data(file_path, filtered=True)

#### Predictions for LGL

In [14]:
predictions_all_toponyms = make_predictions_spacy(data_all_toponyms)
predictions_filtered_toponyms = make_predictions_spacy(data_filtered_toponyms)

588it [00:07, 76.81it/s]
588it [00:07, 81.83it/s]


#### Results LGL & Comparison

In [15]:
# only toponyms w/ lat/long
fps, fns = evaluate(data_filtered_toponyms, predictions_filtered_toponyms)

fp: 1219 | tp: 2027 | fn: 2262
precision: 0.624 | recall: 0.473 | f-score: 0.538


In [20]:
fps

['Orchard St.',
 'the Cottonport Fire Station',
 'Pogemiller',
 'Twin Lakes - Highway',
 'Pelican Rapids',
 'Star Lake - Highway',
 'Highway 55 - Highway 55',
 'County Road',
 'Douglas County Road 4',
 'Lake Mary - Highway',
 'Highway 12',
 'the Red River',
 'Red River',
 'the Red River Valley',
 '40th Avenue South',
 'North Dakota',
 'Oakport Township',
 'Groth',
 'Red River',
 'Fargo City',
 'Red River',
 'Lake Vermont',
 'County Road',
 'County Road',
 'County Road',
 'Nokomis',
 'North Nokomis Street',
 'Darling Avenue',
 'County Road',
 'Van Dorn',
 'Pentagon',
 'New Haven',
 'Sioux Falls',
 'New Haven',
 'the Alexandria City Public Schools',
 'the Durant Center',
 'Old Town',
 'Old Town',
 'Bronx Latin',
 'New York City',
 'Old Town',
 'Northwest D.C.',
 'Cutts',
 'Southern Sudan',
 'Sub-Saharan Africa',
 'Godapitiya',
 'Sri Lanka',
 'Sri Lanka',
 "Sri Lanka's",
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 'Petrojet',
 'Petrojet',
 'Cairo Stadium',
 'Petrojet',
 'Zama

In [21]:
fns

['Rapides Parish',
 'Avoyelles',
 'Cottonport',
 'Alexandria',
 'Mansfield',
 'Mansfield',
 'Shreveport',
 'Mansfield',
 'Mansfield',
 'Cook',
 'Minneapolis',
 'Minnesota',
 'Mahnomen',
 'Twin Lakes',
 'Otter Tail County',
 'Pelican Rapids',
 'Star Lake',
 'Douglas County',
 'Minnesota',
 'Minnesota',
 'Alexandria',
 'Red River',
 'Fargo',
 'Moorhead',
 'Red River',
 'Fargo',
 'Fargo',
 'North Dakota',
 'Oakport Township',
 'Moorhead',
 'Red River',
 'Moorhead',
 'Minnesota',
 'Fargo',
 'Fargo',
 'Red River',
 'Fargo',
 'Douglas County',
 'Lake Vermont',
 'Brandon',
 'Freeborn Lake',
 'Alexandria',
 'Alexandria',
 'Alexandria',
 'America',
 'New Haven',
 'Sioux Falls',
 'S.D.',
 'Surprise',
 'Alexandria',
 'New Haven',
 'Alexandria',
 'Alexandria',
 'Alexandria',
 'New York City',
 'D.C.',
 'D.C.',
 'Sudanese',
 'Sudan',
 'Chinese',
 'Africa',
 'African',
 'Egyptian',
 'Egyptian',
 'Sudan',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Egyptian',
 'Sudanese',
 'Sudanese',
 'Sud

In [16]:
# all toponyms
fps, fns = evaluate(data_all_toponyms, predictions_all_toponyms)

fp: 1180 | tp: 2086 | fn: 2736
precision: 0.639 | recall: 0.433 | f-score: 0.516


In [23]:
fps

['Orchard St.',
 'the Cottonport Fire Station',
 'Pogemiller',
 'Twin Lakes - Highway',
 'Pelican Rapids',
 'Star Lake - Highway',
 'Highway 55 - Highway 55',
 'County Road',
 'Douglas County Road 4',
 'Lake Mary - Highway',
 'Highway 12',
 'the Red River',
 'Red River',
 'the Red River Valley',
 '40th Avenue South',
 'North Dakota',
 'Oakport Township',
 'Groth',
 'Red River',
 'Fargo City',
 'Red River',
 'Lake Vermont',
 'County Road',
 'County Road',
 'County Road',
 'Freeborn Lake',
 'North Nokomis Street',
 'Darling Avenue',
 'County Road',
 'Van Dorn',
 'Pentagon',
 'New Haven',
 'Sioux Falls',
 'New Haven',
 'the Alexandria City Public Schools',
 'the Durant Center',
 'City Hall',
 'Old Town',
 'Old Town',
 'Bronx Latin',
 'New York City',
 'Old Town',
 'Northwest D.C.',
 'Cutts',
 'Southern Sudan',
 'Sub-Saharan Africa',
 'Sri Lanka',
 'Sri Lanka',
 "Sri Lanka's",
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 "Sri Lanka's",
 'Petrojet',
 'Petrojet',
 'Cairo Stadium'

In [24]:
fns

['Rapides Parish',
 'Avoyelles',
 'Cottonport',
 'Memphis St.',
 'Augusta St.',
 'Alexandria',
 'Memphis Street',
 'Harrison',
 'Shamrock Street',
 'Mansfield',
 'Mansfield',
 'Shreveport',
 'Mansfield',
 'Mansfield',
 'DeSoto Parish',
 'Cook',
 'Minneapolis',
 'Minnesota',
 'Highway 200',
 'Mahnomen',
 'Mahnomen County Road',
 'Mahnomen County Road',
 'Twin Lakes',
 'Highway 10',
 'Otter Tail County',
 'Highway 108',
 'Pelican Rapids',
 'Star Lake',
 'Otter Tail/Grant',
 'Grant/Wilkin',
 'County Road 43',
 'Highway 114',
 'Douglas County',
 'Lake Mary',
 'Benson',
 'Minnesota',
 'Minnesota',
 'Alexandria',
 'Red River',
 'Fargo',
 'Moorhead',
 'Red River',
 'Red River Valley',
 'Fargo',
 'Fargo',
 '40th Avenue South',
 'North Dakota',
 'Oakport Township',
 'Moorhead',
 'Red River',
 'Moorhead',
 'Minnesota',
 'Fargo',
 'Fargo',
 'Red River',
 'Fargo',
 'Douglas County',
 'County Road 35',
 'Lake Vermont',
 'County Road 56',
 'County Road 15',
 'Brandon',
 'County Road 96',
 'Freeborn 

## GeoWebNews

In [17]:
# Get file path LGL dataset
file_path = '../../data/GeoWebNews/GeoWebNews.xml'

In [18]:
data_all_toponyms = prepare_data(file_path, filtered=False)
data_filtered_toponyms = prepare_data(file_path, filtered=True)

#### Predictions for GeoWebNews

In [19]:
predictions_all_toponyms = make_predictions_spacy(data_all_toponyms)
predictions_filtered_toponyms = make_predictions_spacy(data_filtered_toponyms)

200it [00:03, 57.99it/s]
200it [00:03, 59.92it/s]


#### Results GeoWebNews & Comparison

In [20]:
# only toponyms w/ lat/long
fps, fns = evaluate(data_filtered_toponyms, predictions_filtered_toponyms)

fp: 425 | tp: 1182 | fn: 1320
precision: 0.736 | recall: 0.472 | f-score: 0.575


In [29]:
fps

['Finiels',
 'the Mississippi River',
 'Press Street',
 'the Chartres/Franklin',
 'Methodist',
 'Royal Street',
 'Royal Street',
 'Royal Street',
 'Desire Street',
 'the Marigny Canal',
 'Washington Square',
 'the Pontchartrain Railroad',
 'Washington Square',
 'Frenchmen Street',
 'Norbury',
 'Granville',
 'Somerville',
 'Lake Manassas',
 'Virginia Gateway',
 'Haymarket Village Center',
 'Old Town',
 'Manassas',
 'Park & Ride',
 'the Vienna/Fairfax-GMU Metro',
 'Dulles International Airport',
 'San Marcos',
 'Shannon Watts',
 'Ruby',
 'Mississippi State’s',
 'Mississippi State',
 'UConn',
 'Mississippi State’s',
 'North Carolina',
 'Mississippi State',
 'Mississippi State',
 'Easter Vigil',
 "St. Peter's Basilica",
 'Bollywood',
 'Manisha Koirala',
 'Fire Island',
 'Fort Salonga',
 'the Easter Vigil',
 'the Manila Cathedral',
 'New Orleans',
 'New York',
 'South Korea',
 'New York City',
 'New Orleans',
 "the Islamic Republic's",
 'Godolphin',
 'West Coast',
 'Godolphin',
 'North Amer

In [30]:
fns

['Louisiana',
 'French',
 'French Quarter',
 'Mississippi River',
 'Faubourg Marigny',
 'German',
 'Irish',
 'Rue d’Enghein',
 'Almonaster',
 'Franklin',
 'Marigny Plantation',
 'Press Street',
 'Faubourg D’Aunoy',
 'Chartres',
 'Franklin',
 'Methodist church',
 'Royal Street',
 'Royal Street',
 'Rue Casa Calvo',
 'Faubourg Marigny',
 '2231 Royal',
 'Greek',
 'Elysian Fields',
 'Royal Street',
 'Bourbon',
 'Desire Street',
 'Bywater',
 'Elysian Fields',
 'New Orleans Railways and Light Company Claiborne Power House',
 'French',
 'Marigny Canal',
 'Washington Square',
 'Champs-Élysées',
 'Elysian Fields',
 'Pontchartrain Railroad',
 'Appalachians',
 'Royal Street',
 'Washington Square',
 'Holy Redeemer Church',
 'Third Presbyterian Church',
 'Frenchmen Street',
 'Carnegie Library',
 'Christopher Inn',
 'Kurdish',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Qamishli',
 'Syrian',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Afrin',
 'Turkish',
 'Afrin',
 'Lake Manassas',
 'Robert Trent Jones Golf Club',

In [21]:
# all toponyms
fps, fns = evaluate(data_all_toponyms, predictions_all_toponyms)

fp: 422 | tp: 1196 | fn: 4565
precision: 0.739 | recall: 0.208 | f-score: 0.324


In [32]:
fps

['Finiels',
 'the Mississippi River',
 'Press Street',
 'the Chartres/Franklin',
 'Methodist',
 'Royal Street',
 'Royal Street',
 'Royal Street',
 'Desire Street',
 'the Marigny Canal',
 'Washington Square',
 'the Pontchartrain Railroad',
 'Washington Square',
 'Frenchmen Street',
 'the Christopher Inn',
 'Lake Manassas',
 'Virginia Gateway',
 'Haymarket Village Center',
 'Old Town',
 'Manassas',
 'Park & Ride',
 'the Vienna/Fairfax-GMU Metro',
 'Dulles International Airport',
 'San Marcos',
 'Shannon Watts',
 'Ruby',
 'Mississippi State’s',
 'Mississippi State',
 'UConn',
 'Mississippi State’s',
 'North Carolina',
 'Mississippi State',
 'Mississippi State',
 'Easter Vigil',
 "St. Peter's Basilica",
 'Bollywood',
 'Manisha Koirala',
 'Fire Island',
 'Fort Salonga',
 'the Easter Vigil',
 'the Manila Cathedral',
 'the Manila Cathedral',
 'New Orleans',
 'New York',
 'South Korea',
 'New York City',
 'New Orleans',
 "the Islamic Republic's",
 'Godolphin',
 'West Coast',
 'Godolphin',
 'No

In [33]:
fns

['area',
 'plantation',
 'mansion',
 'substation',
 'Louisiana',
 'Louisiana Purchase',
 'parcel',
 'French',
 'plat',
 'French Quarter',
 'Mississippi River',
 'squares',
 'neighborhood',
 'city',
 'Faubourg Marigny',
 'community',
 'African Americans',
 'German',
 'Irish',
 'populations',
 'blocks',
 'residents',
 'blocks',
 'intersection',
 'mills',
 'plant',
 'stables',
 'factory',
 'barn',
 'streets',
 'Rue d’Enghein',
 'street',
 'Almonaster',
 'Franklin',
 'avenue',
 'Marigny Plantation',
 'faubourg',
 'Press Street',
 'area',
 'Faubourg D’Aunoy',
 'neighbor',
 'building',
 'Chartres',
 'Franklin',
 'Methodist church',
 'restaurant',
 'street',
 'complex',
 'Royal Street',
 'Royal Street',
 'Rue Casa Calvo',
 'Faubourg Marigny',
 '2231 Royal',
 '2231 Royal',
 'townhouse',
 'basement',
 'Greek',
 'structures',
 'Elysian Fields',
 'Royal Street',
 'Bourbon',
 'Desire Street',
 'Bywater',
 'system',
 'neighborhoods',
 'edifice',
 'Elysian Fields',
 'New Orleans Railways and Light C